In [1]:
%run -p ../train_cnf_disentangle_rl.py --data cifar10 --dims 64,64,64 --strides 1,1,1,1 --num_blocks 2 --layer_type concat --multiscale True --rademacher True --batch_size 8000 --test_batch_size 5000 --save ../experiments_published/cnf_cifar10_bs8K_rl_stdlearnscale_30_annealing_run3 --seed 3 --conditional False --lr 0.01 --warmup_iters 1000 --atol 1e-4  --rtol 1e-4 --scale_fac 1.0 --gate cnn2 --scale_std 15.0 --max_grad_norm 20.0 --annealing_std True
#

/tancode/repos/tan-ffjord/train_cnf_disentangle_rl.py
import argparse
import os
import time
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as tforms
from torchvision.utils import save_image

import lib.layers as layers
import lib.utils as utils
import lib.multiscale_parallel as multiscale_parallel
import lib.modules as modules
import lib.thops as thops

from train_misc import standard_normal_logprob
from train_misc import set_cnf_options, count_nfe, count_parameters, count_total_time, count_nfe_gate
from train_misc import add_spectral_norm, spectral_norm_power_iteration
from train_misc import create_regularization_fns, get_regularization, append_regularization_to_log

from tensorboardX import SummaryWriter

# go fast boi!!
torch.backends.cudnn.benchmark = True
SOLVERS = ["dopri5", "bdf", "rk4", "midpoint", 'adams', 'explicit_adams']
GATES = ["cnn1", "cnn2", "rnn"]

parser = argparse.ArgumentParser("Continuo

Files already downloaded and verified
Files already downloaded and verified


ODENVP(
  (transforms): ModuleList(
    (0): StackedCNFLayers(
      (chain): ModuleList(
        (0): LogitTransform()
        (1): CNF_Gate(
          (gate_net): FeedforwardGateII(
            (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (conv1): Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu1): ReLU(inplace)
            (conv2): Conv2d(10, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu2): ReLU(inplace)
            (avg_layer): AdaptiveAvgPool2d(output_size=(1, 1))
            (linear_layer): Conv2d(10, 2, kernel_size=(1, 1), stride=(1, 1))
            (sigmoid): Sigmoid()
          )
          (odefunc): RegularizedODEfunc(
            (odefunc): ODEf

15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0
15.0


/tancode/repos/tan-ffjord/lib/layers/odefunc_rl.py:291: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  t = torch.tensor(t).type_as(y)
/opt/conda/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Iter 0001 | Time 106.1473(106.1473) | Bit/dim 9.2554(9.2554) | Xent 0.0000(0.0000) | Loss 20.2239(20.2239) | Error 0.0000(0.0000) Steps 478(478.00) | Grad Norm 38.9725(38.9725) | Total Time 0.00(0.00)
Iter 0002 | Time 37.0217(104.0735) | Bit/dim 9.1289(9.2516) | Xent 0.0000(0.0000) | Loss 19.9529(20.2158) | Error 0.0000(0.0000) Steps 472(477.82) | Grad Norm 34.8868(38.8499) | Total Time 0.00(0.00)
Iter 0003 | Time 39.0510(102.1228) | Bit/

...

Iter 1141 | Time 65.9765(65.4529) | Bit/dim 3.5953(3.5780) | Xent 0.0000(0.0000) | Loss 12.5586(9.4204) | Error 0.0000(0.0000) Steps 592(580.97) | Grad Norm 4.3740(3.2763) | Total Time 0.00(0.00)
Iter 1142 | Time 66.6221(65.4880) | Bit/dim 3.6047(3.5788) | Xent 0.0000(0.0000) | Loss 8.9031(9.4049) | Error 0.0000(0.0000) Steps 580(580.94) | Grad Norm 4.4909(3.3127) | Total Time 0.00(0.00)
Iter 1143 | Time 71.6563(65.6730) | Bit/dim 3.6061(3.5796) | Xent 0.0000(0.0000) | Loss 8.9714(9.3919) | Error 0.0000(0.0000) Steps 610(581.81) | Grad Norm 3.2441(3.3107) | Total Time 0.00(0.00)
Iter 1144 | Time 63.0266(65.5936) | Bit/dim 3.6011(3.5803) | Xent 0.0000(0.0000) | Loss 8.5381(9.3663) | Error 0.0000(0.0000) Steps 580(581.75) | Grad Norm 3.0042(3.3015) | Total Time 0.00(0.00)
Iter 1145 | Time 72.4494(65.7993) | Bit/dim 3.5896(3.5805) | Xent 0.0000(0.0000) | Loss 8.8026(9.3494) | Error 0.0000(0.0000) Steps 592(582.06) | Grad Norm 3.5621(3.3093) | Total Time 0.00(0.00)
Iter 1146 | Time 72.2801

9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502
9.281437125748502


Iter 1147 | Time 67.0908(66.0266) | Bit/dim 3.6252(3.5824) | Xent 0.0000(0.0000) | Loss 12.8513(9.4428) | Error 0.0000(0.0000) Steps 592(582.82) | Grad Norm 4.4457(3.3333) | Total Time 0.00(0.00)
Iter 1148 | Time 68.6095(66.1041) | Bit/dim 3.7624(3.5878) | Xent 0.0000(0.0000) | Loss 9.3663(9.4405) | Error 0.0000(0.0000) Steps 628(584.18) | Grad Norm 8.9794(3.5026) | Total Time 0.00(0.00)
Iter 1149 | Time 68.0975(66.1639) | Bit/dim 3.7904(3.5938) | Xent 0.0000(0.0000) | Loss 9.1894(9.4330) | Error 0.0000(0.0000) Steps 646(586.03) | Grad Norm 7.4318(3.6205) | Total Time 0.00(0.00)
Iter 1150 | Time 71.9085(66.3363) | Bit/dim 3.7754(3.5993) | Xent 0.0000(0.0000) | Loss 9.2355(9.4270) | Error 0.0000(0.0000) Steps 646(587.83) | Grad Norm 5.4643(3.6758) | Total Time 0.00(0.00)
Iter 1151 | Time 73.7675(66.5592) | Bit/dim 3.8069(3.6055) | Xent 0.0000(0.0000) | Loss 9.5851(9.4318) | Error 0.0000(0.0000) Steps 712(591.56) | Grad Norm 5.1696(3.7206) | Total Time 0.00(0.00)
Iter 1152 | Time 82.9293

AssertionError: underflow in dt 0.0

         1291112633 function calls (1269795696 primitive calls) in 73079.412 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1152 46402.223   40.280 46402.223   40.280 {method 'run_backward' of 'torch._C._EngineBase' objects}
   260377 20680.210    0.079 20680.210    0.079 {method 'acquire' of '_thread.lock' objects}
   390663  644.802    0.002  644.802    0.002 {method '_write_file' of 'torch._C.CudaFloatStorageBase' objects}
 11136000  564.756    0.000  958.589    0.000 train_cnf_disentangle_rl.py:143(add_noise)
 11136191  354.366    0.000  354.366    0.000 {method 'tobytes' of 'numpy.ndarray' objects}
 11136200  312.446    0.000  312.446    0.000 {method 'uniform_' of 'torch._C._TensorBase' objects}
 11136000  298.320    0.000 1361.241    0.000 functional.py:32(to_tensor)
      988  293.936    0.298  293.936    0.298 {built-in method io.open}
 11136000  220.458    0.000  220.458    0.000 {method 'div' of 'torch._C._